In [1]:
from collect_data import *

In [2]:
meg_path = '/data01/data/MEG'
os.listdir(meg_path)

['index.html?zip=',
 'README.txt',
 'participants.tsv',
 'participants.json',
 'dataset_description.json',
 'sub-11',
 'sub-10',
 'sub-09',
 'sub-08',
 'sub-07',
 'sub-06',
 'sub-05',
 'sub-04',
 'sub-03',
 'sub-02',
 'sub-01',
 'stimuli',
 'meg.gz']

In [3]:
stimuli_path = meg_path + '/stimuli/audio'
wav_files_duration = {}

for filename in os.listdir(stimuli_path):
    if filename.endswith('.wav'): 
        file_path = os.path.join(stimuli_path, filename)
        y, sr = librosa.load(file_path, sr=None)
        duration = librosa.get_duration(y=y, sr=sr)
        wav_files_duration[filename.rstrip('.wav')] = duration
print('WAVE FILES DURATION: ',wav_files_duration)

wav_files_without_numb = {'lw1': 0, 'cable_spool_fort': 1, 'easy_money': 2, 'the_black_willow': 3}
wav_list_without_numb = list(wav_files_without_numb.keys())
print('WAVE FILES WITH\ NUMBERS: ',wav_files_without_numb)



WAVE FILES DURATION:  {'lw1': 318.1481632653061, 'lw1_3': 53.072426303854876, 'lw1_2': 93.7512925170068, 'lw1_1': 75.44276643990929, 'lw1_0': 95.88167800453515, 'easy_money_7': 129.30312925170068, 'easy_money_6': 123.42730158730159, 'easy_money_4': 163.68920634920636, 'easy_money_5': 142.36866213151927, 'easy_money_3': 141.97519274376418, 'easy_money_0': 143.12875283446712, 'easy_money_2': 180.43360544217688, 'easy_money_1': 104.10834467120182, 'cable_spool_fort_4': 123.26390022675737, 'cable_spool_fort_5': 54.69605442176871, 'cable_spool_fort_3': 105.82820861678005, 'cable_spool_fort_2': 137.65614512471655, 'cable_spool_fort_1': 134.8726984126984, 'cable_spool_fort_0': 100.59777777777778, 'the_black_willow_9': 188.23668934240362, 'the_black_willow_8': 142.4142403628118, 'the_black_willow_7': 172.399410430839, 'the_black_willow_6': 128.21628117913832, 'the_black_willow_5': 81.94331065759637, 'the_black_willow_4': 110.45659863945578, 'the_black_willow_2': 125.23859410430839, 'the_black_

In [4]:
bids_path = mne_bids.BIDSPath(
    subject = '01',
    session = '0', 
    task = '0', 
    datatype = "meg", 
    root = meg_path,
)
raw = mne_bids.read_raw_bids(bids_path) 
raw.load_data() # channels X times, no data
df = raw.annotations.to_data_frame()
df = pd.DataFrame(df.description.apply(eval).to_list())

Extracting SQD Parameters from /data01/data/MEG/sub-01/ses-0/meg/sub-01_ses-0_task-0_meg.con...
Creating Raw.info structure...
Setting channel info structure...
Creating Info structure...
Ready.
Reading events from /data01/data/MEG/sub-01/ses-0/meg/sub-01_ses-0_task-0_events.tsv.
Reading channel info from /data01/data/MEG/sub-01/ses-0/meg/sub-01_ses-0_task-0_channels.tsv.
The stimulus channel "STI 014" is present in the raw data, but not included in channels.tsv. Removing the channel.
Reading 0 ... 395999  =      0.000 ...   395.999 secs...


/tmp/ipykernel_577354/314298988.py:8: RuntimeWarning: The unit for channel(s) MISC 001, MISC 002, MISC 003, MISC 004, MISC 005, MISC 006, MISC 007, MISC 008, MISC 009, MISC 010, MISC 011, MISC 012, MISC 013, MISC 014, MISC 015, MISC 016, MISC 017, MISC 018, MISC 019, MISC 020, MISC 021, MISC 022, MISC 023, MISC 024, MISC 025, MISC 026, MISC 027, MISC 028, MISC 029, MISC 030, MISC 031, MISC 032 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


In [5]:
x = raw.get_data()
sampling_freq = raw.info["sfreq"] 
print('RAW_SHAPE: ', x.shape)

RAW_SHAPE:  (256, 396000)


In [6]:
data_meg_chunks = []
meg_from_raw = get_meg_from_raw(raw, data_meg_chunks, sampling_freq)
meg_spectr = get_meg_spectrogram(meg_from_raw)
print('MEG_SPECTR_SHAPE: ', meg_spectr.shape)


/home/matteoc/spect-to-meg/collect_data.py:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /croot/pytorch_1686931851744/work/torch/csrc/utils/tensor_new.cpp:245.)
  data_tensor = torch.tensor(data_meg_chunks)


MEG_SPECTR_SHAPE:  torch.Size([132, 256, 1025, 6])


In [17]:
audio_path = stimuli_path + '/' + wav_list_without_numb[0] + '' + '.wav'
print('AUDIO_PATH: ',audio_path)
audio, sr = librosa.load(audio_path, sr=sampling_audio)
tot_length_audio = audio.shape[0] / sr
print('AUDIO_LENGTH_SAMPLES: ',audio.shape)
print('AUDIO_LENGTH_SECONDS: ',tot_length_audio)

AUDIO_PATH:  /data01/data/MEG/stimuli/audio/lw1.wav
AUDIO_LENGTH_SAMPLES:  (5090371,)
AUDIO_LENGTH_SECONDS:  318.1481875


In [18]:
data_audio_chunks = []
audio_spectr = get_audio_spectrogram(audio_path, tot_length_audio, data_audio_chunks)
print('AUDIO_SPECTR_SHAPE: ', audio_spectr.shape)

AUDIO_SPECTR_SHAPE:  torch.Size([106, 1025, 94])
